# KoBERT를 활용한 고독사 특화 감정 분석

### 필요한 라이브러리 설치

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
!pip install transformers torch scikit-learn pandas numpy

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-i0jpj_gx
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-i0jpj_gx
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Using cached boto3-1.15.18-py2.py3-none-any.whl.metadata (5.3 kB)
  Using cached gluonnlp-0.10.0.tar.gz (344 kB)
  Preparing metadata (setup.py) ... done
  Using cached mxnet-1.7.0.post2-py2.py3-none-manylinux2014_x86_64.whl.metadata (3.0 kB)
INFO: pip is looking at multiple versions of kobert to determine which version is compatible with other requirements. This could take a while.
ERROR: Could not find a version that satisfies the requirement onnxruntime<=1.8.0,==1.8.0 (from kobert) (from versions: 1.12.0, 1.12.1, 1.13.1, 1.14.0, 1.14.1, 1.15.0, 1.15.1, 1.16.0, 1.16.1, 1.16.2, 1.16.3, 1.17.0, 1.17.1, 1.17.3

### KoBERT 모델을 사용한 감정 분석

#### 1. 데이터 준비

In [ ]:
texts = [
    "최근에 아무것도 할 의욕이 없어요. 그냥 침대에 누워만 있고 싶어요. 친구들도 다들 바빠서 저 혼자 남은 것 같아요.",
    "제가 정말 아무 쓸모 없는 사람인 것 같아요. 사람들이 다 나를 이해하지 못하는 것 같아요. 그래서 너무 외로워요.",
    "이상하게 매일매일이 너무 힘들어요. 왜 사는지 모르겠어요. 앞날이 너무 막막해요.",
    "잠을 자도 피곤하고, 일어나면 또 지치고... 하루가 너무 길게 느껴져요. 나 자신에게 확신이 없어요.",
    "예전엔 즐겁던 일들이 이제는 전혀 기쁨이 없어요. 모든 게 무의미하게 느껴져요. 내가 뭘 해야 할지도 모르겠어요."
]


#### 2. 전체 인터뷰 텍스트 분석

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline

# KoBERT 모델 및 토크나이저 불러오기
model_name = "monologg/kobert"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=12)  # 12개의 감정 범주에 맞게 라벨 설정

# 감정 분석 파이프라인 설정
nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# 인터뷰 전체 텍스트 분석
interview_scores = []
for text in texts:
    results = nlp(text)
    scores = {emotion: 0.0 for emotion in range(12)}  # 12개의 감정 범주
    for result in results:
        label = int(result['label'].split('_')[-1])
        score = result['score']
        scores[label] += score
    interview_scores.append(scores)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# 감정 범주별 점수 합산 및 평균 계산
final_scores = {emotion: 0.0 for emotion in range(12)}
for scores in interview_scores:
    for emotion, score in scores.items():
        final_scores[emotion] += score

# 감정 범주별 평균 점수 계산
for emotion in final_scores:
    final_scores[emotion] /= len(interview_scores)

# 감정 이름을 키로 가지는 딕셔너리로 변환
emotion_categories = {
    0: "슬픔",
    1: "흥미 상실",
    2: "무력감",
    3: "자살 생각",
    4: "외로움",
    5: "소외감",
    6: "사회적 고립",
    7: "불안",
    8: "거부감",
    9: "절망감",
    10: "자기 의심",
    11: "자아의 혼란",
    12: "내면의 고독"
}

named_final_scores = {emotion_categories[idx]: score for idx, score in final_scores.items()}


In [ ]:
# 중 카테고리 정의
category_mapping = {
    "우울증적 고독": ["슬픔", "흥미 상실", "무력감", "자살 생각"],
    "사회적 고독": ["외로움", "소외감", "사회적 고립"],
    "정신적 고독": ["불안", "거부감", "절망감"],
    "자아적 고독": ["자기 의심", "자아의 혼란", "내면의 고독"]
}

# 중 카테고리별 점수 합산
category_scores = {category: 0.0 for category in category_mapping.keys()}
for category, emotions in category_mapping.items():
    category_scores[category] = sum(named_final_scores[emotion] for emotion in emotions if emotion in named_final_scores)*100

# 중 카테고리별 감정 분석 결과 출력
print("\n중 카테고리별 감정 분석 결과:")
for category, score in category_scores.items():
    print(f"{category}: {score:.4f}")



중 카테고리별 감정 분석 결과:
우울증적 고독: 9.2574
사회적 고독: 2.1282
정신적 고독: 0.0000
자아적 고독: 0.0000


In [ ]:
# 중 카테고리에 대한 가중치 설정 (필요시 조정 가능)
weights = {
    "우울증적 고독": 4,
    "사회적 고독": 3,
    "정신적 고독": 2,
    "자아적 고독": 1
}

# 가중치 적용하여 최종 점수 계산
weighted_scores = {}
for category, score in category_scores.items():
    weighted_scores[category] = score * weights[category]

# 총점 계산
total_score = sum(weighted_scores.values())

# 위험군 분류
def classify_risk(score):
    if score < 15:
        return "저위험군"
    elif 15 <= score < 30:
        return "중위험군"
    else:
        return "고위험군"

risk_category = classify_risk(total_score)

# 각 카테고리에 대한 주의 메시지 생성
warnings = []
for category, score in category_scores.items():
    if score > 0.4:  # 특정 임계값(예: 0.5) 이상일 경우 경고 메시지 생성
        warnings.append(f"{category}에서 높은 점수({score:.4f})가 나타났습니다. 주의가 필요합니다.")

# 최종 결과 출력
print("\n통합된 중 카테고리별 가중 점수:")
for category, score in weighted_scores.items():
    print(f"{category}: {score:.4f}")

print(f"\n총 점수: {total_score:.4f}")
print(f"위험군 분류: {risk_category}")

# 경고 메시지 출력
if warnings:
    print("\n주의 사항:")
    for warning in warnings:
        print(warning)
else:
    print("\n주의할 점이 없습니다.")



통합된 중 카테고리별 가중 점수:
우울증적 고독: 37.0297
사회적 고독: 6.3847
정신적 고독: 0.0000
자아적 고독: 0.0000

총 점수: 43.4144
위험군 분류: 고위험군

주의 사항:
우울증적 고독에서 높은 점수(9.2574)가 나타났습니다. 주의가 필요합니다.
사회적 고독에서 높은 점수(2.1282)가 나타났습니다. 주의가 필요합니다.
